In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scanpy as sc
from cv2 import resize
from tqdm.notebook import tqdm
import json

## A script to prepare the reference dataset further before axes transfer

In [ ]:
xenium_path = "D:/amonell/timecourse_final/day7_SI_DMSO"

#### You will need the villus segmentation from the aligned IF from day 7 WT to run this. You can find this in the folder ./villus_segmentation

In [ ]:
vill_seg = np.load(os.path.join(xenium_path, "villi_segmentation_mask.npy"))

In [ ]:
adata = sc.read(
    os.path.join(xenium_path, "adatas", "06_before_decomposition_model.h5ad")
)

In [ ]:
file = open(os.path.join(xenium_path, "xenium_output", "experiment.xenium"))
pixel_size = json.load(file)["pixel_size"]

In [ ]:
xvals = adata.obs.x.values
yvals = adata.obs.y.values

downsize_factor = np.load(
    os.path.join(xenium_path, "villi_segmentation_downsize_factor.npy")
)

xvals = (xvals / downsize_factor) * (1 / pixel_size)
yvals = (yvals / downsize_factor) * (1 / pixel_size)

plt.imshow(vill_seg)
plt.scatter(xvals, yvals, s=1)
plt.show()

villi_numbers = vill_seg[yvals.astype(int), xvals.astype(int)]
adata.obs["villi_number"] = villi_numbers

### Defining the crypt-villi axis

In [ ]:
in_villi = []
for i in adata.obs["villi_number"].values:
    if i == 0:
        in_villi.append(False)
    else:
        in_villi.append(True)

In [ ]:
adata.obs["in_villi"] = in_villi
unique_villi = np.unique(adata.obs.villi_number)

In [ ]:
norm_scaled = np.zeros(len(adata.obs.index.values))
for i in tqdm(unique_villi):
    if i == 0:
        indices = np.array(adata.obs["villi_number"] == i)
        norm_vals = adata[indices, :].obs["normalized_crypt_villi"].values
        norm_scaled[indices] = norm_vals
    else:
        indices = np.array(adata.obs["villi_number"] == i)
        norm_vals = adata[indices, :].obs["normalized_crypt_villi"].values
        norm_scaled[indices] = norm_vals / np.max(norm_vals)

In [ ]:
adata.obs["normalized_crypt_villi_scaled"] = norm_scaled

In [ ]:
sc.pl.embedding(
    adata, basis="spatial", color="normalized_crypt_villi_scaled", frameon=False
)

In [ ]:
adata.write(os.path.join(xenium_path, "adatas", "07_axes_defined_reference.h5ad"))